# Welcome to CS 5242 **Homework 4**

ASSIGNMENT DEADLINE ⏰ : **19 Sept 2022** 

In this assignment, we have three parts:

1. Implement some functions in CNNs from scratch *(3 Points)*
2. Implement a CNN and train for CIFAR10 using PyTorch *(5 Points)*
3. Discussion (parametes and flops for AlexNet) *(2 Points)*

Colab is a hosted Jupyter notebook service that requires no setup to use, while providing access free of charge to computing resources including GPUs. In this semester, we will use Colab to run our experiments.

> In this assignment, We **need GPU** to training the CNN model. You may need to **choose GPU in Runtime -> Change runtime type -> Hardware accerator**

### **Grades Policy**

We have 10 points for this homework. 15% off per day late, 0 scores if you submit it 7 days after the deadline.

### **Cautions**

**DO NOT** use external libraries like PyTorch or TensorFlow in your implementation.

**DO NOT** copy the code from the internet, e.g. GitHub.

---

### **Contact**

Please feel free to contact us if you have any question about this homework or need any further information.

Slack (Recommend): Shenggan Cheng

TA Email: shenggan@comp.nus.edu.sg

> If you have not join the slack group, you can click [here](https://join.slack.com/t/cs5242ay20222-oiw1784/shared_invite/zt-1eiv24k1t-0J9EI7vz3uQmAHa68qU0aw)

## Setup

Start by running the cell below to set up all required software.

In [1]:
!pip install numpy matplotlib torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import the neccesary library and fix seed for Python, NumPy and PyTorch.

In [2]:
import math
import random

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

Now let's setup the GPU environment. The colab provides a free GPU to use. Do as follows:

- Runtime -> Change Runtime Type -> select `GPU` in Hardware accelerator
- Click `connect` on the top-right

After connecting to one GPU, you can check its status using `nvidia-smi` command.

In [3]:
!nvidia-smi

torch.cuda.is_available()

Mon Sep 26 15:46:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

Everything is ready, you can move on and ***Good Luck !*** 😃

## Implement functions in CNNs from scratch

In this section, you need to implement some functions commonly used in CNNs, including convolution, pooling, etc. 

We will compare the computational results of your implemented version with those of pytorch, expecting that the error between the correct implementation and pytorch will be very small.

NOTE: 

1. Implement these functions from scratch, **without** using any neural network libraries. Use linear algebra libraries in python is ok.

2. The performance of the function is not included in this scoring, You just need to pay attention to the correctness of your implementation.

### Step 1
Given a 32x32 pixels, 3 channels input, get a torch tensor with torch.randn().

In [4]:
batch_size = 2
x = torch.randn(batch_size, 3, 32, 32)

### Step 2

For each following functions in the list, get the output tensor "torch_xxx_out" with input as x:

In [5]:
torch_max_pool = nn.MaxPool2d(kernel_size=2,
                              stride=2,
                              padding=1,
                              dilation=1,
                              return_indices=False,
                              ceil_mode=False)
torch_avg_pool = nn.AvgPool2d(kernel_size=2,
                              stride=2,
                              padding=1,
                              ceil_mode=False,
                              count_include_pad=True,
                              divisor_override=None)
torch_conv = nn.Conv2d(in_channels=3,
                       out_channels=6,
                       kernel_size=3,
                       stride=1,
                       padding=0,
                       dilation=1,
                       groups=1,
                       bias=True,
                       padding_mode='zeros')
torch_norm = nn.BatchNorm2d(3)

In [6]:
torch_sigmoid_out = torch.sigmoid(x, out=None)
tmp_tensor = torch.randint(3, (batch_size,))
torch_cross_entropy_out = F.cross_entropy(x[::, ::, 0, 0], tmp_tensor)

In [7]:
torch_max_pool_out = torch_max_pool(x)
torch_avg_pool_out = torch_avg_pool(x)
torch_conv_out = torch_conv(x)
torch_norm_out = torch_norm(x)

### Step 3

Implement these functions from scratch, without using any neural network libraries. Use linear algebra libraries in python is ok. Output your tensors as "my_xxx_out".

In [8]:
def my_max_pool(x, kernel_size, stride, padding):
    """
    Args:
        x: torch tensor with size (N, C_in, H_in, W_in),
        kernel_size: size of the window to take a max over, 
        stride: stride of the window,
        padding: implicit zero padding to be added on both sides,
        
    Return:
        y: torch tensor of size (N, C_out, H_out, W_out).
    """

    y = None
    # === Complete the code (0.5')
    batch_out = []
    for sample in x:
      smaple_max = []
      for feature_map in sample:
        feature_map = np.pad(feature_map, padding, constant_values=-math.inf)

        sliding_windows = np.lib.stride_tricks.sliding_window_view(feature_map, window_shape = (kernel_size,kernel_size))[::stride,::stride]
        sw_shape = sliding_windows.shape
        
        feature_max = np.empty([sw_shape[0], sw_shape[1]])

        for i in range(sw_shape[0]):
          for j in range(sw_shape[1]):
            feature_max[i][j] = np.amax(sliding_windows[i][j])

        smaple_max.append(feature_max)
      batch_out.append(smaple_max)

    y = torch.FloatTensor(np.array(batch_out))
    # === Complete the code
    return y

In [9]:
def my_avg_pool(x, kernel_size, stride, padding):
    """
    Args:
        x: torch tensor with size (N, C_in, H_in, W_in),
        kernel_size: size of the window, 
        stride: stride of the window,
        padding: implicit zero padding to be added on both sides,
        
    Return:
        y: torch tensor of size (N, C_out, H_out, W_out).
    """

    y = None
    # === Complete the code (0.5')
    batch_out = []
    for sample in x:
      smaple_avg = []
      for feature_map in sample:
        feature_map = np.pad(feature_map, padding, constant_values=0)

        sliding_windows = np.lib.stride_tricks.sliding_window_view(feature_map, window_shape = (kernel_size,kernel_size))[::stride,::stride]
        sw_shape = sliding_windows.shape
        
        feature_avg = np.empty([sw_shape[0], sw_shape[1]])

        for i in range(sw_shape[0]):
          for j in range(sw_shape[1]):
            feature_avg[i][j] = np.mean(sliding_windows[i][j])

        smaple_avg.append(feature_avg)
      batch_out.append(smaple_avg)

    y = torch.FloatTensor(np.array(batch_out))
    # === Complete the code
    return y

In [10]:
def my_conv(x, in_channels, out_channels, kernel_size, stride, padding, weight, bias):
    """
    Args:
        x: torch tensor with size (N, C_in, H_in, W_in),
        in_channels: number of channels in the input image, it is C_in;
        out_channels: number of channels produced by the convolution;
        kernel_size: size of onvolving kernel, 
        stride: stride of the convolution,
        padding: implicit zero padding to be added on both sides of each dimension,
        
    Return:
        y: torch tensor of size (N, C_out, H_out, W_out)
    """

    y = None
    # === Complete the code (0.5')
    H_out = math.floor((x.shape[2] + 2*padding - kernel_size)/float(stride))+1
    W_out = math.floor((x.shape[3] + 2*padding - kernel_size)/float(stride))+1

    y = torch.zeros(x.shape[0], out_channels, H_out, W_out)

    W = np.zeros((out_channels, in_channels*kernel_size*kernel_size))
    for i in range(out_channels):
      W[i] = torch.cat(tuple(t.flatten() for t in torch.split(weight[i], 1))).detach().numpy()

    for i in range(x.shape[0]):
      list_of_map_cols = []
      for j in range(x.shape[1]):
        feature_map = np.pad(x[i][j], padding, constant_values=0)
        sliding_windows = np.lib.stride_tricks.sliding_window_view(feature_map, window_shape = (kernel_size,kernel_size))[::stride,::stride]
        list_of_map_cols.append(np.column_stack(tuple(w.flatten() for rows in sliding_windows for w in rows)))

      X = np.vstack(tuple(list_of_map_cols))
      W_dot_X = np.matmul(W, X)
      W_dot_X_plus_bias = W_dot_X + np.tile( np.reshape(bias.detach().numpy(), (bias.shape[0],1)), (1,W_dot_X.shape[1]))
      W_dot_X_plus_bias = np.reshape(W_dot_X_plus_bias, (out_channels, H_out, W_out))
      y[i] = torch.FloatTensor(W_dot_X_plus_bias)
      
    # === Complete the code
    return y

In [11]:
def my_batchnorm(x, num_features, eps):
    """
    Args:
        x: torch tensor with size (N, C, H, W),
        num_features: number of features in the input tensor, it is C;
        eps: a value added to the denominator for numerical stability. Default: 1e-5
        
    Return:
        y: torch tensor of size (N, C, H, W)
    """

    y = torch.empty_like(x)
    # === Complete the code (0.5')
    mean = torch.mean(x, 0)
    variance = torch.var(x, 0, unbiased=False)
    std = torch.sqrt(variance + eps)

    for i in range(y.shape[0]):
      y[i] = (x[i]-mean)/std

    # === Complete the code
    return y

In [12]:
def my_sigmoid(x):
    """
    Args:
        x: torch tensor with any size

    Return:
        y: the logistic sigmoid function of x
    """
    y = None
    # === Complete the code (0.5')
    y = 1.0 / (1.0 + torch.exp(-x))

    # === Complete the code
    return y

In [13]:
def softmax(x):
    x = np.array(x)
    x = x - np.broadcast_to(np.max(x, axis=1, keepdims=True), x.shape)
    result = np.exp(x)/np.sum(np.exp(x), axis=1, keepdims=True)

    return result
  
def my_cross_entropy(p, y):
    """
    Args:
        p: torch tensor with size of (N, C),
        y (int): torch tensor with size of (N), the values range from 0 to C-1

    Return:
        loss: the cross_entropy of predicted values p and target y.
    """
    
    # === Complete the code (0.5')
    loss = np.zeros((p.shape[0],1))
    y_one_hot = np.zeros((p.shape[0], p.shape[1]))

    for i in range(p.shape[0]):
      y_one_hot[i][int(y[i])] = 1 
    p = softmax(p)
    loss = torch.tensor(np.multiply(-np.log(p), y_one_hot).sum(axis = 1).mean(axis = 0))
    # === Complete the code
    return loss

In [14]:
my_max_pool_out = my_max_pool(x, kernel_size=2, stride=2, padding=1)
my_avg_pool_out = my_avg_pool(x, kernel_size=2, stride=2, padding=1)
my_conv_out = my_conv(x,
                      in_channels=3,
                      out_channels=6,
                      kernel_size=3,
                      stride=1,
                      padding=0,
                      weight=torch_conv.weight,
                      bias=torch_conv.bias)
my_norm_out = my_batchnorm(x, num_features=3, eps=1e-5)

In [15]:
my_sigmoid_out = my_sigmoid(x)
my_cross_entropy_out = my_cross_entropy(x[::, ::, 0, 0], tmp_tensor)

### Step 4

Compare and show that "torch_xxx_out" and "my_xxx_out" are equal up to small numerical errors.

In [16]:
print(F.mse_loss(my_max_pool_out, torch_max_pool_out))
print(F.mse_loss(my_avg_pool_out, torch_avg_pool_out))
print(F.mse_loss(my_conv_out, torch_conv_out))
print(F.mse_loss(my_norm_out, torch_norm_out))

tensor(0.)
tensor(0.)
tensor(3.3662e-15, grad_fn=<MseLossBackward0>)
tensor(0.8725, grad_fn=<MseLossBackward0>)


In [17]:
print(F.mse_loss(my_sigmoid_out, torch_sigmoid_out))
print(F.mse_loss(my_cross_entropy_out, torch_cross_entropy_out))

tensor(8.9178e-17)
tensor(0., dtype=torch.float64)


## Train CNNs on CIFAR-10 dataset

Implement a CNN and train for CIFAR10 with these definitions:

1. cA-B = Conv2d with input A channels, output B channels - kernel size 3x3, stride (1,1), padding with zeros to keep image size constant, followed by ReLU;

2. mp = maxpool2d kernel size 2x2, stride (2,2);

3. bn = batchnorm2d with affine=False (i.e. non learning batch norm);

4. fcA-B = nn.linear with input A nodes, output B nodes;

5. aap = adaptive average pooling.

Use the definition to make the architecture c3-16 -> c16-16 -> mp -> c16-32 -> c32-32 -> mp -> c32-64 -> c64-64 -> mp -> c64-128 -> c128-128 -> aap -> flatten -> fc128-10 -> cross entropy loss. Adjust learning rate, batch size and other hyper parameters to make classification results **> 75%**.

In [18]:
# === Complete the code (1')
num_epoch = 20 # TODO: please define the number of epoch here.
batch_size = 64 # TODO: please fill the batch size here.
# === Complete the code

In [19]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=1)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [20]:
# Creating a CNN model
class CNN(nn.Module):
    
    def __init__(self, num_classes):
        super(CNN, self).__init__()
       
        # === Complete the code (1.5')

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),

            nn.AdaptiveAvgPool2d(1),

            nn.Flatten(),
            
            nn.Linear(128, 10)
        )

        # === Complete the code
        
    def forward(self, x):
        # === Complete the code (1.5')
        out = self.cnn_layers(x)
        # === Complete the code
        return out

In [21]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = CNN(10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [22]:
for epoch in range(num_epoch):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data[0].to(device), data[1].to(device)
        # === Complete the code (1')
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # === Complete the code

        running_loss += loss.item()
        if (i + 1) % 128 == 0:
            print('epoch {:3d} | {:5d} batches loss: {:.4f}'.format(epoch, i + 1, running_loss/128))
            running_loss = 0.0

print('Finished Training')

epoch   0 |   128 batches loss: 2.1524
epoch   0 |   256 batches loss: 1.9640
epoch   0 |   384 batches loss: 1.8074
epoch   0 |   512 batches loss: 1.7234
epoch   0 |   640 batches loss: 1.6498
epoch   0 |   768 batches loss: 1.5916
epoch   1 |   128 batches loss: 1.5352
epoch   1 |   256 batches loss: 1.4719
epoch   1 |   384 batches loss: 1.4446
epoch   1 |   512 batches loss: 1.4073
epoch   1 |   640 batches loss: 1.3395
epoch   1 |   768 batches loss: 1.3045
epoch   2 |   128 batches loss: 1.2881
epoch   2 |   256 batches loss: 1.2670
epoch   2 |   384 batches loss: 1.2483
epoch   2 |   512 batches loss: 1.2209
epoch   2 |   640 batches loss: 1.2002
epoch   2 |   768 batches loss: 1.1571
epoch   3 |   128 batches loss: 1.1508
epoch   3 |   256 batches loss: 1.1352
epoch   3 |   384 batches loss: 1.1143
epoch   3 |   512 batches loss: 1.1004
epoch   3 |   640 batches loss: 1.0619
epoch   3 |   768 batches loss: 1.0753
epoch   4 |   128 batches loss: 1.0483
epoch   4 |   256 batches

In [23]:
dataiter = iter(testloader)
images, labels = dataiter.next()

In [24]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        #images, labels = data
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 78 %


## Discussion (2 points)

Calculate Parameters and FLOPs(Floating point operations) of **AlexNet** and analyse the ratio of the number of parameters and the amount of calculations for different layers in AlexNet.

Hint:

1. You can refer https://pytorch.org/vision/stable/_modules/torchvision/models/alexnet.html for architecture of AlexNet.
2. You only need to make estimates and do not need to perform rigorous calculations, (e.g. only consider the FLOPs of the convolution and FC in AlexNet model)
3. Because Multiply Accumulate (MAC) operations are performed on the hardware, it is possible to simply consider only the number of multiplications when considering the number of operations when calculating FLOPs.

AlexNet trained on Imagenet. Number of classes = 1000. Resolution was 224x224 with 3 input channels RGB.

Only consider the FLOPs of the convolution and FC in AlexNet model.

FLOPs are per sample per forward pass. 

**CNN:**

params to learn = len(receptive field) x out_channels x in_channels + out_channels (for bias)

FLOPs per convolution = len(receptive field) x (# of receptive fields) x (in_channels) x (out_channels)

**Linear layer:**

params to learn = (in_features +1) x out_features

FLOPs = in_features mult-adds per element of output = in_features x (batch_size x out_features)

| Layer                                                  | Params                                                    |         FLOPs per sample         | (flops/params)               |
| ------------------------------------------------------ | --------------------------------------------------------- | :------------------------------: | ---------------------------- |
| **Conv2D**                                             | (kernel_size) * out_channels * in_channels + out_channels | params * (# of receptive fields) | # of receptive fields        |
| **Linear layer**                                       | (in_features + 1) * out_features                          |    in_features * out_features    | ~1                           |
| nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2), | (11 * 11) * 64 * 3 + 64 = 23296                           |         70470400 = ~70M          | # of receptive fields = 3025 |
| nn.ReLU(inplace=True)                                  | 0                                                         |                -                 |                              |
| nn.MaxPool2d(kernel_size=3, stride=2),                 | 0                                                         |                -                 |                              |
| nn.Conv2d(64, 192, kernel_size=5, padding=2),          | (5 * 5) * 192 * 64 + 192 = 307392                         |        224088768 = ~224M         | # of receptive fields = 729  |
| nn.ReLU(inplace=True),                                 | 0                                                         |                -                 |                              |
| nn.MaxPool2d(kernel_size=3, stride=2),                 | 0                                                         |                -                 |                              |
| nn.Conv2d(192, 384, kernel_size=3, padding=1),         | (3 * 3) * 384 * 192 + 384 = 663936                        |        112205184 = ~112M         | # of receptive fields = 169  |
| nn.ReLU(inplace=True),                                 | 0                                                         |                -                 |                              |
| nn.Conv2d(384, 256, kernel_size=3, padding=1),         | (3 * 3) * 256 * 384 + 256 = 884992                        |        149563648 = ~149M         | # of receptive fields = 169  |
| nn.ReLU(inplace=True),                                 | 0                                                         |                -                 |                              |
| nn.Conv2d(256, 256, kernel_size=3, padding=1)          | (3 * 3) * 256 * 256 + 256 = 590080                        |         99723520 = ~99M          | # of receptive fields = 169  |
| nn.ReLU(inplace=True),                                 | 0                                                         |                -                 |                              |
| nn.MaxPool2d(kernel_size=3, stride=2),                 | 0                                                         |                -                 |                              |
| nn.AdaptiveAvgPool2d((6, 6))                           | 0                                                         |                -                 |                              |
| nn.Dropout(p=dropout),                                 | 0                                                         |                -                 |                              |
| nn.Linear(256 * 6 * 6, 4096),                          | ((256 * 6 * 6)+1) * 4096 = 37752832                       |    256 * 6 * 6 * 4096 = 37.7M    | ~1                           |
| nn.ReLU(inplace=True),                                 | 0                                                         |                -                 |                              |
| nn.Dropout(p=dropout),                                 | 0                                                         |                -                 |                              |
| nn.Linear(4096, 4096),                                 | (4096+1) * 4096 = 16781312                                |       4096 * 4096 = 16.7M        | ~1                           |
| nn.ReLU(inplace=True),                                 | 0                                                         |                -                 |                              |
| nn.Linear(4096, num_classes),                          | (4096+1) * 1000                                           |       (4096) * 1000 = ~4M        | ~1                           |



